In [1]:
import torch
import numpy as np 
import pandas as pd
from statistics import fmean
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from steamgrid import SteamGridDB
from steamgrid import StyleType, PlatformType, MimeType, ImageType
import requests
import os
import tqdm

In [2]:
file_path = "/media/lars/F77E-7D07/"
AuthKey = "afe8e25264becc810a0af3258a099804"

In [3]:
locations = pd.read_csv("filtered_archive/games_locations.csv")

In [4]:
locations[["title", "app_id", "location_2d"]]

,title,app_id,location_2d
0,Prince of Persia: Warrior Within™,13500,[-27.830263 -50.389065]
1,Monaco: What's Yours Is Mine,113020,[-23.66106 -32.13113]
2,Escape Dead Island,226560,[-21.555323 -15.31565 ]
3,METAL SLUG 3,250180,[-21.135366 -32.897854]
4,Enclave,253980,[-25.930891 -7.152575]
...,...,...,...
2663,Toodee and Topdee,1303950,[ 8.808064 -12.508147]
2664,Strategic Mind: Blitzkrieg,1200330,[29.4271 51.273785]
2665,Burnhouse Lane,1846460,[-14.998375 -3.0924897]
2666,RiffTrax: The Game,1707870,[ 6.7055907 31.31667 ]


In [5]:
sgdb = SteamGridDB(AuthKey)

In [7]:
steamid = 13500
id = sgdb.get_game_by_steam_appid(steamid).id
grid = sgdb.get_grids_by_gameid([id])

In [8]:
url = ""
for i in range(len(grid)):
    if grid[i].height == 900 and grid[i].width == 600:
        url = grid[i].thumbnail
        break

if url == "":
    url = grid[0].thumbnail

In [6]:
import os
import requests

failed_list = []
for i in range(len(locations)):
    print(i / len(locations))
    steamid = int(locations.iloc[i]["app_id"])
    
    # Construct the file path
    file_path_2 = file_path + "thumbnails/" + str(steamid) + ".jpg"
    
    # Skip downloading if the file already exists
    if os.path.exists(file_path_2):
        continue

    try:
        id = sgdb.get_game_by_steam_appid(steamid).id
        grid = sgdb.get_grids_by_gameid([id])

        if grid:
            url = ""
            for item in grid:
                if item.height == 900 and item.width == 600:
                    url = item.thumbnail
                    break

            if not url:
                url = grid[0].thumbnail

            # Create the directory if it doesn't exist
            os.makedirs(os.path.dirname(file_path_2), exist_ok=True)

            # Send a GET request to the URL
            response = requests.get(url)

            # Check if the request was successful
            if response.status_code == 200:
                # Save the image to a file
                with open(file_path_2, "wb") as file:
                    file.write(response.content)
            else:
                print(f"Failed to download image. Status code: {response.status_code}, Steamid: {steamid}")
                failed_list.append(steamid)
        else:
            failed_list.append(steamid)

    except Exception as e:
        print(f"Error processing SteamID {steamid}: {e}")
        failed_list.append(steamid)


0.0
0.0003748125937031484
0.0007496251874062968
0.0011244377811094452
0.0014992503748125937
0.0018740629685157421
0.0022488755622188904
0.002623688155922039
0.0029985007496251873
0.003373313343328336
0.0037481259370314842
0.004122938530734633
0.004497751124437781
0.004872563718140929
0.005247376311844078
0.005622188905547227
0.005997001499250375
0.006371814092953523
0.006746626686656672
0.00712143928035982
0.0074962518740629685
0.007871064467766116
0.008245877061469266
0.008620689655172414
0.008995502248875561
0.009370314842578711
0.009745127436281859
0.010119940029985007
0.010494752623688156
0.010869565217391304
0.011244377811094454
0.011619190404797601
0.01199400299850075
0.012368815592203899
0.012743628185907047
0.013118440779610194
0.013493253373313344
0.013868065967016492
0.01424287856071964
0.01461769115442279
0.014992503748125937
0.015367316341829085
0.015742128935532233
0.016116941529235384
0.01649175412293853
0.01686656671664168
0.017241379310344827
0.017616191904047975
0.0179

In [7]:
locations[locations["app_id"].isin(failed_list)]

,Unnamed: 0,app_id,title,location,location_2d
20,20,1872790,Luckitown,[-0.54882944 -0.1049142 -0.03180152 0.062386...,[ 2.5503721 -7.788539 ]
66,66,542770,Castle Must Be Mine,[-0.00950029 0.12249824 0.10713808 0.164986...,[-21.076326 18.85381 ]
81,81,758980,Poker Legends: Texas Hold'em Poker Tournaments,[-0.0540731 -0.02135447 0.04075522 -0.030828...,[ 1.8751603 19.730494 ]
85,85,672630,Academia : School Simulator,[ 0.02466989 -0.04786646 -0.03815473 -0.017808...,[-6.4131403 -6.901415 ]
92,92,544390,NITE Team 4 - Military Hacking Division,[ 0.03309607 -0.00515141 -0.04086332 0.014254...,[-9.161272 -5.1755853]
...,...,...,...,...,...
2651,2651,2067850,Drakensang Online,[ 0.02311709 -1.8571211 0.13721025 0.879054...,[ 2.889666 49.3656 ]
2653,2653,789100,My Little Farmies,[-0.00772906 0.12380558 -0.10354359 0.198945...,[-9.978387 22.277626]
2658,2658,1188680,The Political Machine 2020,[ 0.03437734 0.05475251 0.09848739 -0.039261...,[ 8.319886 32.743477]
2660,2660,1263370,Seek Girl:Fog Ⅰ,[-0.0073267 -0.08888978 0.04604013 0.203427...,[-26.03432 -14.17556]


In [8]:
failed = pd.DataFrame(failed_list,columns=["app_id"])
failed.to_csv(file_path + "failed.csv")

In [9]:
failed = pd.read_csv(file_path + "failed.csv")

In [10]:
locations_with_images = locations[~locations["app_id"].isin(failed["app_id"])][["title", "app_id", "location_2d"]]

In [11]:
locations_with_images[['x', 'y']] = locations_with_images['location_2d'].str.strip('[]').str.split(expand=True).astype(float)

In [12]:
locations_with_images = locations_with_images.drop(["location_2d"], axis=1)

In [13]:
locations_with_images['image_path'] = "thumbnails/" + locations_with_images["app_id"].astype(str) + ".jpg"

In [14]:
locations_with_images.to_csv(file_path + "locations_with_images.csv")

In [15]:
locations_with_images

,title,app_id,x,y,image_path
0,Prince of Persia: Warrior Within™,13500,-27.830263,-50.389065,thumbnails/13500.jpg
1,Monaco: What's Yours Is Mine,113020,-23.661060,-32.131130,thumbnails/113020.jpg
2,Escape Dead Island,226560,-21.555323,-15.315650,thumbnails/226560.jpg
3,METAL SLUG 3,250180,-21.135366,-32.897854,thumbnails/250180.jpg
4,Enclave,253980,-25.930891,-7.152575,thumbnails/253980.jpg
...,...,...,...,...,...
2663,Toodee and Topdee,1303950,8.808064,-12.508147,thumbnails/1303950.jpg
2664,Strategic Mind: Blitzkrieg,1200330,29.427100,51.273785,thumbnails/1200330.jpg
2665,Burnhouse Lane,1846460,-14.998375,-3.092490,thumbnails/1846460.jpg
2666,RiffTrax: The Game,1707870,6.705591,31.316670,thumbnails/1707870.jpg


In [16]:
locations_with_images["w"] = 0
locations_with_images["h"] = 0

In [17]:
from PIL import Image

for index, row in locations_with_images.iterrows():
    image = Image.open(file_path + row["image_path"])
    w, h = image.size
    locations_with_images.at[index, "w"] = w 
    locations_with_images.at[index, "h"] = h 


In [18]:
locations_with_images["h"].value_counts()

h
400    2519
196      14
Name: count, dtype: int64

In [19]:
locations_with_images.to_csv(file_path + "locations_with_images.csv")